In [3]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [4]:
import pickle
import pandas as pd

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [7]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
year = 2021
month = 2

In [8]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [10]:
y_pred.mean()

16.191691679979066

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [12]:
df_results = pd.DataFrame()
df_results['ride_id'] = df['ride_id']
df_results['duration_prediction'] = y_pred

In [15]:
output_file=f'tmp/fhv_tripdata_{year:04d}_{month:02d}.parquet'

In [16]:
df_results.to_parquet(output_file, engine = 'pyarrow', compression = None, index = False)